### SIEM

In [ ]:
import requests
import time
import re
from dataclasses import dataclass
from typing import Generator, Optional
from io import IOBase

In [ ]:
def tail_follow(file: IOBase) -> Generator[str, None, None]:
    file.seek(0, 2)
    while True:
        line = file.readline()
        if line:
            yield line
            continue
        time.sleep(1)

In [ ]:
def papertrail_tail(token: str) -> Generator[str, None, None]:
    url = "https://papertrailapp.com/api/v1/events/search.json"
    headers = {"X-Papertrail-Token": token}

    tail = False
    min_id = None
    limit = 10000

    while True:
        params = {"limit": limit}
        if tail and min_id:
            params["min_id"] = min_id
        else:
            params["min_time"] = int(time.time()) - 10
        
        r = requests.get(url, params=params, headers=headers)
        r.raise_for_status()
        data = r.json()
        tail = data.get("tail", False)
        min_id = data.get("max_id", None)
        yield from (e["message"] for e in data["events"])
        time.sleep(5)

In [ ]:
@dataclass
class PacketInfo:
    """ Information about a network packet"""
    timestamp: str
    protocol: str
    src: str
    dst: str
    data: str
    

In [ ]:
# TODO Syslog format parsing?

def parse_tcpdump(line: str) -> Optional[PacketInfo]:
    pattern = re.compile(r'(\d+:\d+:\d+\.\d+) (\w+) ([\d\.]+|\w+) > ([\d\.]+|\w+): (.+)$')
    match = pattern.match(line)
    if match:
        return PacketInfo(*list(match.groups()))
    return None

In [ ]:
for line in papertrail_tail("YOUR_TOKEN_HERE"):
    pkt = parse_tcpdump(line)
    if not pkt:
        print("Packet format error")
    else:
        print(f"Packet from {pkt.src} to {pkt.dst}: {pkt.data}")